In [11]:
import os, itertools

import numpy as np
import meshio
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans, DBSCAN
from sklearn.metrics.pairwise import pairwise_distances_argmin
import networkx as nx

def plot_3d(points):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.scatter(points[:,0],points[:,1],points[:,2])

In [18]:
n_surfaces = 2
surface = r'X:\20220113_surface_sampling_samples\buckinghorse\EM_BD_PW01-2in-90-D01_3_1_1.stl'
mesh = meshio.read(surface)
points = mesh.points
triangles = mesh.cells_dict['triangle']

v1v0 = np.array([points[tri_i[1]] - points[tri_i[0]] for tri_i in triangles])
v2v0 = np.array([points[tri_i[2]] - points[tri_i[0]] for tri_i in triangles])
normals = np.cross(v1v0,v2v0,axisa=1,axisb=1)
normals /= np.linalg.norm(normals,axis=1)[:,np.newaxis]
centroids = np.mean(points[triangles],axis=1)
centroids -= np.mean(centroids,axis=0)
centroids /= (np.max(centroids,axis=0)-np.min(centroids,axis=0))
X = np.hstack([centroids,normals])

kmeans = MiniBatchKMeans(
    n_clusters=n_surfaces,
    batch_size=50
)
print("Fitting")
kmeans.fit(X)
cluster_centers = kmeans.cluster_centers_
mbk_labels = pairwise_distances_argmin(X,cluster_centers)

Fitting


In [19]:
# Initial triangles found
# Need to clean up stray triangles and put surfaces together
print("Cleaning up clusters")
cluster_bounds = []
for cluster in range(n_surfaces):
    cluster_triangles = triangles[cluster == mbk_labels]
    #https://stackoverflow.com/questions/61584283/find-connected-components-in-list-of-triangle-vertices
    edges = []
    for face in cluster_triangles:
        edges.extend(list(itertools.combinations(face,2)))
    
    graph = nx.from_edgelist(edges)

    components = list(nx.algorithms.components.connected_components(graph))
    component_to_faces = dict()
    component_triangles = dict()
    for component in components:
        # component_to_faces[tuple(component)] = [face for face in cluster_triangles if set(face) <= component]
        component_triangles[tuple(component)] = [i for i,face in enumerate(cluster_triangles) if set(face) <= component]
    # components = [[face for face in cluster_triangles if set(face) <= component ] for component in components]
    triangle_groups = sorted(component_triangles.values(),key=lambda x: len(x),reverse=True)
    print(f"{len(cluster_triangles)-len(triangle_groups[0])} triangles of {len(cluster_triangles)} to be relocated")
    cluster_idx = np.where(cluster == mbk_labels)[0]
    for triangle_group in triangle_groups[1:]:
        mbk_labels[cluster_idx[triangle_group]] = -1

    cluster_bounds.append(np.vstack([
        np.min(centroids[cluster == mbk_labels],axis=0),
        np.max(centroids[cluster == mbk_labels],axis=0)]))

relocations = np.where(mbk_labels == -1)[0]
for t_i, t in enumerate(relocations):
    for k in range(n_surfaces):
        if np.any(np.greater(centroids[t],cluster_bounds[k][1])):
            continue
        if np.any(np.less(centroids[t],cluster_bounds[k][0])):
            continue
        mbk_labels[t] = k
        break

Cleaning up clusters
96 triangles of 577351 to be relocated
2605 triangles of 502069 to be relocated


In [20]:
print("Splitting surfaces")
for k in range(n_surfaces):
    print(k)
    cluster_triangles = triangles[k == mbk_labels]
    cluster_points = points[cluster_triangles]
    point_reindex,cluster_triangles = np.unique(cluster_triangles,return_inverse=True)
    cluster_points = points[point_reindex]
    cluster_triangles = np.reshape(cluster_triangles,(-1,3))
    mesh = meshio.Mesh(cluster_points,{'triangle':cluster_triangles})
    folder = os.path.dirname(surface)
    file = os.path.basename(surface)
    file_split = os.path.splitext(file)
    mesh.write(os.path.join(folder,f"{file_split[0]}_{k}.stl"),file_format="stl-binary")

Splitting surfaces
0
1
